In [ ]:
pip install requests beautifulsoup4 markdownify

In [ ]:
import os

import requests
from bs4 import BeautifulSoup as bs
import markdownify
import time

blog_url = 'https://hyostech.tistory.com'
image_save_dir = 'assets/images/posts'
start_page_no = 5
end_page_no = 148

os.makedirs('_posts', exist_ok=True)
for page_no in range(start_page_no, end_page_no + 1):
    print(page_no, ' ... ', end='')
    html = requests.get(f'{blog_url}/{page_no}')

    if html.status_code == 404:
        print('404')
        continue

    soup = bs(html.text, "html.parser")

    title = soup.select_one(".article-header .title-article")
    print(title.text, ' ... ', end='')

    content = soup.select_one(".article-view .contents_style")
    content_md = markdownify.markdownify(str(content), heading_style="ATX")
 
    category = soup.select_one(".article-header .category")
    
    date = soup.select_one(".article-header .date")
    year, month, day, _ = date.text.split('.')
 
    tags = soup.select(".article-tag .box-tag a")
    tags = '\n  '.join(list(map(lambda tag: f'- {tag.text}', tags)))

    md_base_name = f'{year.strip()}-{month.strip()}-{day.strip()}-tistory-post-{page_no}'
    image_dir = os.path.join(image_save_dir, md_base_name)

    if content.select("img"):
        os.makedirs(image_dir, exist_ok=True)
    for n, image in enumerate(content.select("img"), start=1):
        src = image['src']
        image_path = os.path.join(image_dir, f'img-{n}.png')
        open(image_path, 'wb').write(requests.get(src).content)
        content_md = content_md.replace(src, f'/{image_path}')

    metadata = f"""---
title: {title.text.replace(':', '&#58;')}
toc: true
categories:
  - {category.text}
tags:
  {tags}
---

"""
    content_md = metadata + content_md
    open(os.path.join('_posts', f'{md_base_name}.md'), 'w').write(content_md)

    print('OK')